In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
import cufflinks as cf
cf.go_offline()
import chart_studio as cs
from IPython.display import display, HTML
import seaborn as sns
import yfinance as yf
from datetime import datetime, date
%matplotlib inline

/home/matteo/snap/jupyter/common/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [17]:
#estrapolo i dati  per trovare la combinazione di portafoglio adeguata

start_date = '2013-01-01'
end_date = '2019-01-01'
capitale = 100
assets= ['T', 'WMB', 'IRM', 'MO', 'PM','D','PGR','GIL','IBM','VZ', 'PPL', 'K','DUK', 'PFE','PSA',
       'IP', 'SO','KMI','XOM','WMT']

#Dataframe per valori di chiusura
pf_data = pd.DataFrame()
#Dataframe per i dividendi
date_today = datetime.now()
days = pd.date_range(start_date, end_date , freq='D')
pf_div = pd.DataFrame(index = days)

#Riempio il dataframe pf_data con le diverse stock
for a in assets:
    pf_data[a]= wb.DataReader(a,data_source='yahoo', start = start_date , end= end_date)['Adj Close']

#Riempio il dataframe pf_div con i dividendi delle stock
for a in range(len(assets)):
    
    stock = yf.Ticker(assets[a])
    pf_div[assets[a]]=stock.dividends
    

pf_div = pf_div.dropna(how='all') #cancella le righe che sono tutte NaN
pf_div.fillna(0, inplace=True) # trasmormo Nan in 0 per fare calcoli
        

In [28]:
#calcolo ritorno logaritmico gioraliero degli assets mel periodo selezionato
log_returns =np.log(pf_data/pf_data.shift(1))
#percentuale di ritorno annua media di ogni stocks
mean_annual_return = round(log_returns.mean()*250*100,1)
#percentuale di volatilità annua media di ogni stock
mean_annual_volatility = round(log_returns.std()*250**0.5*100,1) 
#valore dei dividendi medio annua per ogni stock
mean_annual_dividend = round((pd.DataFrame(pf_div.resample('Y').sum())).mean(),2)
# rapporto tra prezzo stock e i suoi dividendi
dividends_stock = round((mean_annual_dividend / pf_data.values[len(pf_data)-1])*100,2)### valore di ieri
# PROBLEMA se prendo un anno ancora non completo mi si sfancula tutto
# IPOTESI FORTE prendo solo dati da inzio anno fino a fine anno
div = pf_div.resample('Y').sum()
tasso_crescita = ((div-div.shift(1))/div)
tasso_crescita = tasso_crescita.iloc[1:]# cancello prima riga fatta di nan
tasso_return = round(tasso_crescita.mean()*100,2)
tasso_vol = round(tasso_crescita.std()*100**0.5,2)


#Creo Dataframe per un review veloce
review = pd.DataFrame(index = assets)
review['Mean annual Return [%]'] = mean_annual_return
review['Mean annual Volatility [%]'] = mean_annual_volatility
review['Mean annual Dividends [$/N°stock]'] = mean_annual_dividend
review['Dividends / Stock price'] = dividends_stock
review['Tasso annuo crescita dividendi [%]'] = tasso_return
review['Tasso annuo volatilità dividendi [%]'] = tasso_vol
review

,Mean annual Return [%],Mean annual Volatility [%],Mean annual Dividends [$/N°stock],Dividends / Stock price,Tasso annuo crescita dividendi [%],Tasso annuo volatilità dividendi [%]
T,1.8,16.9,1.90,7.30,2.09,0.01
WMB,-2.0,44.0,1.68,8.29,-5.48,3.46
IRM,7.7,24.9,2.05,6.97,7.71,2.50
MO,11.3,17.5,2.32,5.15,9.26,0.34
PM,0.3,18.5,4.22,6.82,3.46,1.57
D,8.8,16.0,2.74,4.06,7.59,0.10
PGR,19.1,17.2,0.69,1.20,20.44,2.95
GIL,9.9,23.1,0.39,1.31,1.69,2.07
IBM,-5.5,19.7,5.54,5.16,8.44,1.77
VZ,8.5,16.8,2.23,4.22,2.64,0.06


## Possibile inserire un filtro che seleziona solo se si avverano determinate condizioni?  
  esempio:
* rit > 5%
* vol < 25%

# Analisi sui dividendi
* Dividendo in crescita > inflazione ~2%
* Payout Ratio < 0.65 : dividendi non superiori al 65% utili, deve reinvestire per svilupparsi 
* Utili in crescita: in modo che il divedendo non vada a erodere le riserve 
* Rendimento > 3% : dividends/stock price(odierno) > 3% 
* debito/equity < 1,5: per evitare che i debiti creino problemi sui dividendi 
* beta: meglio se inferiore a S&P (=1), se è più alto vuol dire che è più volatile 